# Postgres Embedding | Postgres の組み込み

> [Postgres Embedding](https://github.com/neondatabase/pg_embedding)は、`Postgres`用のオープンソースベクター類似性検索であり、近似最近傍探索に`Hierarchical Navigable Small Worlds (HNSW)`を使用しています。
>
> > [Postgres Embedding](https://github.com/neondatabase/pg_embedding) is an open-source vector similarity search for `Postgres` that uses  `Hierarchical Navigable Small Worlds (HNSW)` for approximate nearest neighbor search.

> 以下をサポートしています：
>
> > It supports:
>
> * HNSWを使用した正確な最近傍探索と近似最近傍探索

>   > exact and approximate nearest neighbor search using HNSW

> * L2距離

>   > L2 distance


このノートブックは、Postgresベクターデータベース（`PGEmbedding`）の使用方法を示しています。

> This notebook shows how to use the Postgres vector database (`PGEmbedding`).

> PGEmbedding統合によってpg\_embedding拡張機能が作成されますが、それをPostgresに追加するためには以下のクエリを実行する必要があります：
>
> > The PGEmbedding integration creates the pg\_embedding extension for you, but you run the following Postgres query to add it:

```sql
CREATE EXTENSION embedding;
```




In [ ]:
# Pip install necessary package
!pip install openai
!pip install psycopg2-binary
!pip install tiktoken

`OpenAIEmbeddings`を使用するためには、環境変数にOpenAI APIキーを追加してください。

> Add the OpenAI API Key to the environment variables to use `OpenAIEmbeddings`.




In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:········


In [3]:
## Loading Environment Variables
from typing import List, Tuple

In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGEmbedding

In [5]:
os.environ["DATABASE_URL"] = getpass.getpass("Database Url:")

Database Url:········


In [6]:
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
connection_string = os.environ.get("DATABASE_URL")
collection_name = "state_of_the_union"

In [13]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)

query = "What did the president say about Ketanji Brown Jackson"
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

## Working with vectorstore in Postgres | Postgresでのvectorstoreの扱い方




### Uploading a vectorstore in PG | PGにベクターストアをアップロードする




In [32]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
    pre_delete_collection=False,
)

### Create HNSW Index | HNSWインデックスを作成する

デフォルトでは、この拡張機能は100%のリコールを持つシーケンシャルスキャン検索を実行します。`similarity_search_with_score`の実行時間を短縮するために、近似最近傍探索（ANN）用のHNSWインデックスの作成を検討してください。ベクトル列にHNSWインデックスを作成するには、`create_hnsw_index`関数を使用します。

> By default, the extension performs a sequential scan search, with 100% recall. You might consider creating an HNSW index for approximate nearest neighbor (ANN) search to speed up `similarity_search_with_score` execution time. To create the HNSW index on your vector column, use a `create_hnsw_index` function:




In [ ]:
PGEmbedding.create_hnsw_index(
    max_elements=10000, dims=1536, m=8, ef_construction=16, ef_search=16
)

上記の関数は、以下のSQLクエリを実行することと同等です：

> The function above is equivalent to running the below SQL query:

```sql
CREATE INDEX ON vectors USING hnsw(vec) WITH (maxelements=10000, dims=1536, m=3, efconstruction=16, efsearch=16);
```

上記のステートメントで使用されているHNSWインデックスのオプションには以下が含まれます：

> The HNSW index options used in the statement above include:

* maxelements: インデックスされる要素の最大数を定義します。これは必須のパラメータです。上に示された例では、値は3です。実際の例では、はるかに大きな値、例えば1000000のような値が設定されます。「要素」とはデータセット内のデータポイント（ベクトル）のことで、HNSWグラフ内のノードとして表されます。通常、このオプションにはデータセットの行数を収容できるような値を設定します。

  > maxelements: Defines the maximum number of elements indexed. This is a required parameter. The example shown above has a value of 3. A real-world example would have a much large value, such as 1000000. An "element" refers to a data point (a vector) in the dataset, which is represented as a node in the HNSW graph. Typically, you would set this option to a value able to accommodate the number of rows in your in your dataset.


* dims: ベクトルデータの次元数を定義します。これは必須パラメータです。上記の例では小さい値が使用されています。例えば、1536次元をサポートするOpenAIのtext-embedding-ada-002モデルを使用して生成されたデータを格納する場合、1536という値を設定します。

  > dims: Defines the number of dimensions in your vector data. This is a required parameter. A small value is used in the example above. If you are storing data generated using OpenAI's text-embedding-ada-002 model, which supports 1536 dimensions, you would define a value of 1536, for example.


* m: グラフ構築中に各ノードに対して作成される双方向リンク（「エッジ」とも呼ばれます）の最大数を定義します。
  以下の追加のインデックスオプションがサポートされています：

  > m: Defines the maximum number of bi-directional links (also referred to as "edges") created for each node during graph construction.
  > The following additional index options are supported:


* efConstruction: インデックス構築時に考慮される最も近い隣接点の数を定義します。デフォルト値は32です。

  > efConstruction: Defines the number of nearest neighbors considered during index construction. The default value is 32.


* efsearch: インデックス検索時に考慮される最近傍の数を定義します。デフォルト値は32です。これらのオプションをどのように設定することでHNSWアルゴリズムに影響を与えるかについては、[HNSWアルゴリズムのチューニング](https://neon.tech/docs/extensions/pg_embedding#tuning-the-hnsw-algorithm)を参照してください。

  > efsearch: Defines the number of nearest neighbors considered during index search. The default value is 32.
  > For information about how you can configure these options to influence the HNSW algorithm, refer to [Tuning the HNSW algorithm](https://neon.tech/docs/extensions/pg_embedding#tuning-the-hnsw-algorithm).





### Retrieving a vectorstore in PG | PG内でベクターストアを取得する




In [15]:
store = PGEmbedding(
    connection_string=connection_string,
    embedding_function=embeddings,
    collection_name=collection_name,
)

retriever = store.as_retriever()

In [16]:
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.pghnsw.HNSWVectoreStore object at 0x121d3c8b0>, search_type='similarity', search_kwargs={})

In [17]:
db1 = PGEmbedding.from_existing_index(
    embedding=embeddings,
    collection_name=collection_name,
    pre_delete_collection=False,
    connection_string=connection_string,
)

query = "What did the president say about Ketanji Brown Jackson"
docs_with_score: List[Tuple[Document, float]] = db1.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)